In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ssanj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ssanj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
data = {
"review": [
"I love this movie! It is amazing and wonderful",
"This film was terrible. I hated every moment",
"What a fantastic experience, absolutely loved it!",
"Worst movie ever, waste of time",
"The story was good but acting was bad"
],
"sentiment": [1, 0, 1, 0, 0]
}
df = pd.DataFrame(data)
print(df)

                                              review  sentiment
0     I love this movie! It is amazing and wonderful          1
1       This film was terrible. I hated every moment          0
2  What a fantastic experience, absolutely loved it!          1
3                    Worst movie ever, waste of time          0
4              The story was good but acting was bad          0


In [17]:
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)         # keep only letters
    tokens = text.split()                        # simple split instead of nltk.word_tokenize
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

In [19]:
df['clean_review'] = df['review'].apply(preprocess_text)
print("\nPreprocessed Reviews:\n", df['clean_review'])


Preprocessed Reviews:
 0             love movi amaz wonder
1    film terribl hate everi moment
2       fantast experi absolut love
3         worst movi ever wast time
4                stori good act bad
Name: clean_review, dtype: object


In [21]:
max_words = 1000
max_len = 20
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['clean_review'])
sequences = tokenizer.texts_to_sequences(df['clean_review'])

In [23]:
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(df['sentiment'])
print("\nTokenized & Padded:\n", X)


Tokenized & Padded:
 [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5  6  7  8  9]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10 11 12  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 13  2 14 15 16]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 17 18 19 20]]


In [27]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=16))
model.add(SimpleRNN(16))
model.add(Dense(1, activation='sigmoid'))

In [35]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=2, verbose=1)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 1.0000 - loss: 0.5192
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.4864
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.4623
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.4389
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.4168
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.3913
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.3662
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.3417
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.3180
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.2924
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.2692
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.2460
E

In [37]:
test_sentence = "The movie was awesome and fantastic"
test_clean = preprocess_text(test_sentence)
test_seq = tokenizer.texts_to_sequences([test_clean])
test_pad = pad_sequences(test_seq, maxlen=max_len)

In [39]:
prediction = model.predict(test_pad)[0][0]
print("\nTest Review:", test_sentence)
print("Cleaned Review:", test_clean)
print("Predicted Sentiment:", "Positive 😊" if prediction > 0.5 else "Negative 😞")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step

Test Review: The movie was awesome and fantastic
Cleaned Review: movi awesom fantast
Predicted Sentiment: Positive 😊
